# CNN LSTM

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.preprocessing import Normalizer

import numpy as np
import pandas as pd

In [2]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [3]:
traindata = pd.read_csv(DATA+'kddcup.data.corrected.csv', header=None)
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [4]:
from utils import preprocess
traindata,testdata=preprocess.preprocess(traindata)

In [5]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [6]:
scaler = Normalizer().fit(X)
X_train = scaler.transform(X)
X_test = scaler.transform(T)

In [7]:
y_train= Y
y_test= C

In [8]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [9]:
model = Sequential()
model.add(LSTM(5, activation="sigmoid", input_shape=X_train.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(23, activation="softmax"))

epoch=0

In [10]:
# define optimizer and objective, compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5)                 140       
_________________________________________________________________
dropout (Dropout)            (None, 5)                 0         
_________________________________________________________________
dense (Dense)                (None, 20)                120       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 23)                4

In [12]:
# train
epoch+=1
print(epoch)
model.fit(X_train, y_train, epochs=1, validation_split=0.3, shuffle=True, batch_size=1024)

1
2512/2512 [==============================] - 48s 18ms/step - loss: 0.8467 - accuracy: 0.6831 - val_loss: 0.0947 - val_accuracy: 0.9847


In [13]:
model.evaluate(X_test, y_test,batch_size=1024)

1196/1196 [==============================] - 8s 6ms/step - loss: 0.1417 - accuracy: 0.9862


[0.14167387783527374, 0.9861841797828674]

In [14]:
model.save(MODELS+"lstm_model.hdf5")